In [2]:
import os

In [3]:
%pwd

'd:\\MLpipeline\\mlpipeline-example\\research'

In [4]:
# to create artifcats inside root folder , that is mlpipeline-example
# move one folder back

os.chdir("../")

In [5]:
%pwd

'd:\\MLpipeline\\mlpipeline-example'

In [56]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path # variable : data type
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
# the above are the same things we defined in artifacts

In [57]:
# entity is return type of function 
# for example, return as config Box


# update configuration manager (config/config.py)
# it will return all the file path , source URL at one shot

# after read , read the 3 yaml files stored in constants/__init__.py


In [68]:
# import 

from src.mlProject.constants import *
# whatever things are inside constants import everything

from src.mlProject.utils.common import read_yaml,create_directories


In [69]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [70]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [71]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [72]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()


[2024-12-22 12:28:31,759: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-22 12:28:31,762: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-22 12:28:31,768: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-22 12:28:31,770: INFO: common: created directory at: artifacts]
[2024-12-22 12:28:31,774: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-22 12:28:31,777: INFO: 3447005916: File already exists of size: ~ 26 KB]
